In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
from torch.utils.data.sampler import SubsetRandomSampler
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        # (((178−2)/2)−2)/2=43
        # (((218−2)/2)−2)/2=53
        self.fc1 = nn.Linear(43*53*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 43*53*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

# A1model = torch.load('./A1/A1RegnetY16GFModel.pt')
A2model = ConvolutionalNetwork()
A2model.load_state_dict(torch.load('./A2/A2CNNModelFinal.pt'))
# A2model = torch.load('./A2/A2CNNModelFinal.pt')
# B1model = torch.load('./B2/B2CNNModelFinal.pt')
# B2model = torch.load('./B1/B2CNNModelFinal.pt')


<All keys matched successfully>

In [3]:
test_transform = transforms.Compose([
        # Requirements for pretrained models:
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [8]:
from torch.utils.data import Dataset
from PIL import Image

class MyDataset(Dataset):
    def __init__(self, task, part, root_dir, label_file, class_names, class_to_idx, transform):
        self.root_dir = root_dir
        self.label_file = label_file
        self.class_names = class_names
        self.class_to_idx = class_to_idx
        self.transform = transform

        # Read the labels from the Excel file
        if (task == 'A' and part == 1): 
            self.labels = pd.read_csv('./Datasets/celeba_test/labels.csv', usecols=[2], delimiter='\t')['gender'].tolist()
        if (task == 'A' and part == 2):
            self.labels = pd.read_csv('./Datasets/celeba_test/labels.csv', usecols=[3], delimiter='\t')['smiling'].tolist()
        if (task == 'B' and part == 1): 
            self.labels = pd.read_csv('./Datasets/cartoon_set_test/labels.csv', usecols=[2])['face_shape'].tolist()
        if (task == 'B' and part == 2):
            self.labels = pd.read_csv('./Datasets/cartoon_set_test/labels.csv', usecols=[1])['eye_color'].tolist()
        
        # Get the list of image files (without the lambda it was not in order)
        self.image_files = []
        for root, dirs, files in os.walk(root_dir): 
            self.image_files.extend([os.path.join(root, file) for file in sorted(files, key=lambda x: int(''.join(filter(str.isdigit, x))))])


    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image and apply the transformation
        image = Image.open(self.image_files[idx])
        if self.transform:
            image = self.transform(image)

        # Convert the label to a tensor
        label = torch.tensor(self.labels[idx])
        
        return image, label

# Define the root directory of the dataset and the label file
root_dir_test = './Datasets/celeba_test/img'
label_file_test = './Datasets/cartoon_set_test/img'
# Define the class names and their corresponding indexes
class_names = ['men', 'women']
class_to_idx = {'men': 1, 'women': -1}

A1test_data = MyDataset('A', 1, root_dir_test, label_file_test, class_names, class_to_idx, transform=test_transform)
A2test_data = MyDataset('A', 2, root_dir_test, label_file_test, class_names, class_to_idx, transform=test_transform)
B1test_data = MyDataset('B', 1, root_dir_test, label_file_test, class_names, class_to_idx, transform=test_transform)
B2test_data = MyDataset('B', 2, root_dir_test, label_file_test, class_names, class_to_idx, transform=test_transform)

torch.manual_seed(42)

A1test_loader = DataLoader(A1test_data, batch_size=125, shuffle=True)
A2test_loader = DataLoader(A2test_data, batch_size=125, shuffle=True)
B1test_loader = DataLoader(B1test_data, batch_size=125, shuffle=True)
B2test_loader = DataLoader(B2test_data, batch_size=125, shuffle=True)

# A1


In [6]:
test_losses = []
test_correct = []
predictions = []

# Run the testing batches
with torch.no_grad():
    for b, (X_test, y_test) in enumerate(A2test_loader):
        # Changing the labels to be in the range [0, num_classes-1].
        y_test = torch.where(y_test == 1, torch.tensor(1), torch.tensor(0))

        # Apply the model
        y_val = A2model(X_test)

        # Tally the number of correct predictions
        predicted = torch.max(y_val.data, 1)[1]
        predictions.append(predicted)

        

AttributeError: 'MyDataset' object has no attribute 'labels'